In [29]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

In [1]:

# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp_ner = spacy.load('en_core_web_sm', disable=['parser'])

'''sentence = "The striped bats are hanging on their feet for best"

# Parse the sentence using the loaded 'en' model object `nlp`
doc = nlp(sentence)

# Extract the lemma for each token and join
" ".join([token.lemma_ for token in doc])
#> 'the strip bat be hang on -PRON- foot for good''''

'the stripe bat be hang on their foot for good'

In [23]:
df=pd.read_csv('model_2_results_positive_from30000samples.csv')

In [38]:
def preprocess(df):
    
    smell_words=["Stink","Scent","Smell","Odour","Odor","Stench","Reek","Aroma","Aromatic","Whiff","Foetor","Fetor","Fragrance","Musk","Rankness","Redolence","Pong","Pungency","Niff","Deodorant","Essence","Atmosphere","Olfaction",
                               "Stinking","Stank","Stunk","Scented","Odourless","Odoriferous","Odorous","Malodorous","Reeking","Aromatic","Whiffy","Fetid","Foetid","Fragrant","Fragranced","Redolent","Frowzy","Frowsy","Pungent","Funky","Musty","Niffy","Unscented","Scentless","Deodorized","Noisome","Smelly","Mepehitic","Muddle","Putrid","Olfactory","Musky","Pungently",
                               "Smelling","Smelled","Reeked","Sniff","Sniffed","Sniffing","Whiffed","Deodorized","Deodorizing","Snuffing","Snuffed"]
    smell_words_lower=[x.lower() for x in smell_words]

    my_words=['love', 'good', 'new','stink', 'stank',"n't", 'ca', "'s", "gon", 'na', 'll', 't', 'ur', 'bc', 'nt', 'yo', 've', 'ai','lol','cause']

    # stop = stopwords.words('english')
    s= set(ENGLISH_STOP_WORDS).union(smell_words).union(my_words).union(smell_words_lower) #.union(set(stop))
    
    #taking off links
    df['Text']=df['sentence'].apply(lambda x: re.split('http*:\/\/.*', str(x))[0])

    # make everything lower case, Not done for proper nouns
    # df['Text'] = df['Text'].str.lower()

    #take off stop words
    df['Text_clean'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (s)]))
    
    # only take a lsit of clean texts, since that is what the moel needs as input : a list
    docs = list(df['Text_clean'].loc[:].values)
    
    # lemmatization
    
    '''nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    lem_token= [nlp(sentence) for sentence in docs] 
    lem_doc= [" ".join([token.lemma_ for token in tok]) for tok in lem_token]
    clean_lem = [' '.join([word for word in row.split() if word not in (s)]) for row in lem_doc]
    
    clean_lem[:5]'''
    return docs
    



In [68]:
dfc=preprocess(df)

In [77]:
len(dfc)

19474

In [135]:
dfc[:5][0].


<function str.index>

In [145]:
docu = [[nlp_ner(sentence),sentence] for sentence in dfc]

In [146]:

entities=pd.DataFrame([], columns=['text','label','explanation'])

for sentence in docu:
    for word in sentence[0].ents:
        entities = entities.append({
            'sentence': sentence[1],
            'text': word.text,
            'label': word.label_,
            'explanation': spacy.explain(word.label_)
        }, ignore_index=True)


In [160]:
entities.head(10)

,text,label,explanation,sentence
0,Tomorrow,DATE,Absolute or relative dates or periods,Tomorrow @ 12 EST! Hsuan's book aesthetics inc...
1,Hsuan,PERSON,"People, including fictional",Tomorrow @ 12 EST! Hsuan's book aesthetics inc...
2,Organic Rose Water Organic Aloe Vera Gel,ORG,"Companies, agencies, institutions, etc.",Hydrate Toner moisturizing ingredients like Or...
3,Mandalay,MONEY,"Monetary values, including unit",Soon residents downtown #Mandalay lit candles ...
4,#militarycoup,MONEY,"Monetary values, including unit",Soon residents downtown #Mandalay lit candles ...
5,Myanmar,GPE,"Countries, cities, states",Soon residents downtown #Mandalay lit candles ...
6,#,CARDINAL,Numerals that do not fall under another type,Soon residents downtown #Mandalay lit candles ...
7,today,DATE,Absolute or relative dates or periods,"Can sell set today? Lime, mandarin basil wax m..."
8,TB,ORG,"Companies, agencies, institutions, etc.","Can sell set today? Lime, mandarin basil wax m..."
9,p*ssy p*ssy loyal,MONEY,"Monetary values, including unit",pray dirty house educated hoe job gold digger ...


In [165]:
valuecounts= entities[entities.label == 'GPE']['text'].value_counts()

In [166]:
valuecounts[:10]

⠀           174
US           37
UK           20
Texas        16
Miami        16
China        16
Ireland      14
DC           14
Scotland     14
America      14
Name: text, dtype: int64

In [168]:
with pd.option_context("display.min_rows", 50, "display.max_rows", 200, "display.max_columns", 15, 'display.max_colwidth', 200):
    display(entities[entities.text == 'Texas']['sentence'])

2420                                                Texas friends, care mentally week. Today’s announcement absolute hot, garbage feel little hopeless today. Take break, don’t doom scroll, let breathe little.
2561                          House member Rep Roy Texas today’s hearing hate speech crimes bragging 700 lynchings TX did people color. A tall tree idea cool. This racist go. You help ALL people district hate
3034     Rep. Ronny Jackson, R-Texas, sexual denigrating statements female subordinate, alcohol duty, humiliated staff long stint White House physician, scathing inspector general report finds. https://kpb...
4556                        Tasty Texas wine pair w spicy prawn n chorizo jambalaya! @mcphersoncellar Tri Colore Red treat - medium-dry, dark grapes & nicely rugged texture-yum! #texaswine #winesndfoodpairing
7047                                                                                I things JFK dad called said got tickets 6th Floor Museum Saturday. We plans vis